In [39]:
import pandas as pd
from sqlalchemy import create_engine

# Defino la ruta del archivo CSV
csv_file_path = r"C:\Users\USER\Downloads\dataset\Urban Air Quality and Health Impact Dataset.csv"

# Cargo el dataset
df = pd.read_csv(csv_file_path)

# Parámetros de conexión
user = 'postgres'
password = '1234'
host = 'localhost'  
port = '5432'       
database = 'postgres'

# Creo la cadena de conexión
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# Cargo el DataFrame a la base de datos
df.to_sql('nombre_tabla', engine, if_exists='replace', index=False)  # Cambia 'nombre_tabla' 
print("Datos cargados exitosamente a la base de datos.")



Datos cargados exitosamente a la base de datos.


In [40]:
# 1. Obtengo todos los registros
all_data = pd.read_sql("SELECT * FROM nombre_tabla", engine)
print(all_data)


       datetime  datetimeEpoch     tempmax    tempmin       temp  \
0    2024-09-07   1.725692e+09  106.100000  91.000000  98.500000   
1    2024-09-08   1.725779e+09  103.900000  87.000000  95.400000   
2    2024-09-09   1.725865e+09  105.000000  83.900000  94.700000   
3    2024-09-10   1.725952e+09  106.100000  81.200000  93.900000   
4    2024-09-11   1.726038e+09  106.100000  82.100000  94.000000   
..          ...            ...         ...        ...        ...   
995  2024-09-18   1.726633e+09   76.060546  64.359387  69.002142   
996  2024-09-17   1.726550e+09   68.409198  65.939319  66.567410   
997  2024-09-12   1.726122e+09   69.756690  65.286919  65.919492   
998  2024-09-14   1.726284e+09   77.106797  61.481724  68.106569   
999  2024-09-18   1.726618e+09   90.923080  79.296868  81.636991   

     feelslikemax  feelslikemin  feelslike        dew   humidity  ...  \
0      104.000000     88.100000  95.900000  51.500000  21.000000  ...   
1      100.500000     84.700000  92.3

In [41]:
# 2. Obtengo registros con temperatura máxima mayor a 100°F
hot_days = pd.read_sql("SELECT * FROM nombre_tabla WHERE tempmax > 100", engine)
print(hot_days)



      datetime  datetimeEpoch     tempmax    tempmin       temp  feelslikemax  \
0   2024-09-07   1.725692e+09  106.100000  91.000000  98.500000    104.000000   
1   2024-09-08   1.725779e+09  103.900000  87.000000  95.400000    100.500000   
2   2024-09-09   1.725865e+09  105.000000  83.900000  94.700000     99.900000   
3   2024-09-10   1.725952e+09  106.100000  81.200000  93.900000    100.600000   
4   2024-09-11   1.726038e+09  106.100000  82.100000  94.000000    101.000000   
..         ...            ...         ...        ...        ...           ...   
74  2024-09-08   1.725810e+09  102.769151  87.318951  95.107769     99.365629   
75  2024-09-09   1.725876e+09  104.003776  83.243193  97.109865    101.090371   
76  2024-09-17   1.726589e+09  102.556340  87.681657  93.321286     98.415744   
77  2024-09-10   1.725977e+09  105.402624  82.106679  95.330343    101.152571   
78  2024-09-13   1.726175e+09  101.889047  78.491983  87.953773     99.644005   

    feelslikemin  feelslike

In [42]:
# 3. Número de días en cada temporada
season_counts = pd.read_sql("""
   SELECT "Season", COUNT(*) as Count
FROM nombre_tabla
GROUP BY "Season";
""", engine)
print(season_counts)



  Season  count
0   Fall   1000


In [43]:
# 4. Obtengo el promedio de la humedad por ciudad
avg_humidity = pd.read_sql("""
    SELECT "City", AVG(humidity) as Average_Humidity
    FROM nombre_tabla
    GROUP BY "City"
""", engine)
print(avg_humidity)



            City  average_humidity
0        Chicago         52.980994
1        Houston         70.770566
2    San Antonio         59.352298
3    Los Angeles         57.071243
4       San Jose         60.942343
5        Phoenix         20.956228
6         Dallas         56.280617
7  New York City         61.998322
8   Philadelphia         61.878699
9      San Diego         72.003168


In [44]:
# 5. Filtro días de fin de semana con riesgo para la salud mayor a 10
weekend_health_risk = pd.read_sql("""
    SELECT *
    FROM nombre_tabla
    WHERE "Is_Weekend" = TRUE AND "Health_Risk_Score" > 10
""", engine)
print(weekend_health_risk)

       datetime  datetimeEpoch     tempmax    tempmin       temp  \
0    2024-09-07   1.725692e+09  106.100000  91.000000  98.500000   
1    2024-09-08   1.725779e+09  103.900000  87.000000  95.400000   
2    2024-09-15   1.726384e+09  100.100000  82.100000  92.000000   
3    2024-09-07   1.725685e+09   92.000000  73.400000  81.700000   
4    2024-09-14   1.726290e+09   92.900000  76.700000  84.400000   
..          ...            ...         ...        ...        ...   
125  2024-09-07   1.725631e+09   88.230138  70.750230  76.808929   
126  2024-09-08   1.725822e+09   84.503260  76.016346  80.997397   
127  2024-09-07   1.725715e+09   85.914371  69.609476  76.683118   
128  2024-09-15   1.726329e+09   96.272981  76.779528  85.587411   
129  2024-09-21   1.726914e+09   95.695752  83.288149  90.238724   

     feelslikemax  feelslikemin  feelslike        dew   humidity  ...  \
0      104.000000     88.100000  95.900000  51.500000  21.000000  ...   
1      100.500000     84.700000  92.3